In [3]:
import gensim
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import nltk
from nltk import word_tokenize
from nltk import sent_tokenize
import re
import os

In [10]:
books = pd.read_csv(r'D:/books.csv', sep=';', error_bad_lines = False, encoding='latin-1')
users = pd.read_csv(r'D:/users.csv', sep=';',error_bad_lines = False, encoding='latin-1')
ratings = pd.read_csv(r'D:/rating.csv',  sep=';',error_bad_lines = False, encoding='latin-1')

C:\Users\moaaz\AppData\Local\Temp/ipykernel_10608/1070825346.py:1: FutureWarning: The error_bad_lines argument has been deprecated and will be removed in a future version. Use on_bad_lines in the future.


  books = pd.read_csv(r'D:/books.csv', sep=';', error_bad_lines = False, encoding='latin-1')
b'Skipping line 6452: expected 8 fields, saw 9\nSkipping line 43667: expected 8 fields, saw 10\nSkipping line 51751: expected 8 fields, saw 9\n'
b'Skipping line 92038: expected 8 fields, saw 9\nSkipping line 104319: expected 8 fields, saw 9\nSkipping line 121768: expected 8 fields, saw 9\n'
b'Skipping line 144058: expected 8 fields, saw 9\nSkipping line 150789: expected 8 fields, saw 9\nSkipping line 157128: expected 8 fields, saw 9\nSkipping line 180189: expected 8 fields, saw 9\nSkipping line 185738: expected 8 fields, saw 9\n'
b'Skipping line 209388: expected 8 fields, saw 9\nSkipping line 220626: expected 8 fields, saw 9\nSkipping line 227933: expected 8 fields, saw 11\nSkipping line 2289

In [11]:
users.shape

(278858, 3)

In [13]:
books.shape

(271360, 8)

In [28]:
ratings.shape

(1149780, 3)

In [86]:
ratings['User-ID'].unique().shape  # A total of 105283 users gave rating.

(105283,)

In [270]:
temp = ratings['User-ID'].value_counts() >  1 # Let's take out the user ids who rated more than 200 books
processed= temp[temp]  # Users who gave 1more than 200 views

In [271]:
processed.shape  # Only 899 users gave more than 200 reviews

(46117,)

In [272]:
indices = processed.index   # Let's store the indices of the users who gave more than 200 ratings

In [273]:
final_ratings=ratings[ratings['User-ID'].isin(indices)]

In [274]:
final_ratings.head()  # Dataframe of users who gave more than 200 ratings

,User-ID,ISBN,Book-Rating
3,276729,052165615X,3
4,276729,0521795028,6
10,276746,0425115801,0
11,276746,0449006522,0
12,276746,0553561618,0


In [275]:
final_ratings.shape

(1090614, 3)

In [276]:
ratings_with_name = final_ratings.merge(books, on='ISBN')


In [277]:
no_of_ratings_for_each_book = ratings_with_name.groupby('Book-Title')[['Book-Rating']].count().reset_index()
no_of_ratings_for_each_book = no_of_ratings_for_each_book.rename(columns = {"Book-Rating":"No of rating"})

In [278]:
no_of_ratings_for_each_book

,Book-Title,No of rating
0,"A Light in the Storm: The Civil War Diary of Amelia Martin, Fenwick Island, Delaware, 1861 (Dear America)",4
1,Always Have Popsicles,1
2,Apple Magic (The Collector's series),1
3,"Ask Lily (Young Women of Faith: Lily Series, Book 5)",1
4,Beyond IBM: Leadership Marketing and Finance for the 1990s,1
...,...,...
236965,Ã?Â?lpiraten.,2
236966,Ã?Â?rger mit Produkt X. Roman.,4
236967,Ã?Â?sterlich leben.,1
236968,Ã?Â?stlich der Berge.,3


In [310]:
gg = no_of_ratings_for_each_book.merge(ratings_with_name, on='Book-Title')

In [311]:
gg=gg[gg['No of rating']>=20]

In [312]:
gg.shape

(410535, 11)

In [313]:
gg=gg.drop_duplicates(['User-ID', 'Book-Title'])

In [314]:
gg.shape

(406528, 11)

In [315]:
pt = gg.pivot_table(index='Book-Title',columns='User-ID',values='Book-Rating')


In [316]:
pt=pt.fillna(0)

In [317]:
pt

User-ID,8,9,10,14,16,17,26,32,39,44,...,278798,278818,278832,278838,278843,278844,278846,278849,278851,278854
Book-Title,,,,,,,,,,,,,,,,,,,,,
'Salem's Lot,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
01-01-00: The Novel of the Millennium,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
10 Lb. Penalty,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1001 Ways to Be Romantic,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
101 Dalmatians,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
"\O\"" Is for Outlaw""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
"\Surely You're Joking, Mr. Feynman!\"": Adventures of a Curious Character""",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
e,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [318]:
from scipy.sparse import csr_matrix

In [319]:
pt_sparse = csr_matrix(pt)

In [320]:

pt_sparse

<7223x34905 sparse matrix of type '<class 'numpy.float64'>'
	with 151371 stored elements in Compressed Sparse Row format>

In [321]:
from sklearn.neighbors import NearestNeighbors as NN

In [322]:
model = NN(algorithm='brute')

In [323]:
model.fit(pt_sparse)

NearestNeighbors(algorithm='brute')

In [335]:
def recommend(book_name):
    id = np.where(pt.index ==book_name)[0][0]
    distances, suggestions = model.kneighbors(pt.iloc[id,:].values.reshape(1,-1), n_neighbors=6)
    for i in range(len(suggestions)):
            
        print(pt.index[suggestions[i]])

In [336]:
recommend("Little Women")

Index(['Little Women', 'False Witness', 'Evening Star', 'Hot Flashes',
       'BODILY HARM', 'The Silver Rose'],
      dtype='object', name='Book-Title')
